In [75]:
import numpy as np
from scipy.fftpack import dct, idct
from scipy.fft import fft, ifft
import pandas as pd
import matplotlib.pyplot as plt

def fdm(X, fs, fc, filter_type='dct', sort_fc='ascend', plot_subbands=True):
    N = X.shape[0]
    fc = np.sort(fc)
    if fc[0] != 0:
        fc = np.hstack((0, fc))
    if fc[-1] != fs / 2:
        fc = np.hstack((fc, fs / 2))
    if sort_fc == 'descend':
        fc = fc[::-1]

    if filter_type == 'dct':
        dct_type = 2
        K = np.round(2 * N * fc / fs).astype(int)
        no_of_subbands = K.shape[0] - 1
        Hk = np.zeros((N, 1, no_of_subbands))

        for i in range(no_of_subbands):
            if sort_fc == 'ascend':
                Hk[K[i]:K[i+1], :, i] = 1
            if sort_fc == 'descend':
                Hk[K[i+1]:K[i], :, i] = 1

        Xk = dct(X, type=dct_type, n=N, axis=0, norm='ortho')
        Yk = np.einsum('ij,ijk->ijk', Xk, Hk)
        Y = idct(Yk, type=dct_type, n=N, axis=0, norm='ortho')

    if plot_subbands:
        t = np.arange(0, X.shape[0]) / fs
        plt.figure(figsize=(16, 12))
        for k in range(no_of_subbands):
            plt.subplot(no_of_subbands, 1, k + 1)
            plt.plot(t, Y[:, 0, k])
            plt.title(f"Sub-band {k + 1}: {fc[k]}-{fc[k + 1]} Hz")
        plt.tight_layout()
        plt.show()

    return Y

# Main script
file_path = 'synthetic_eeg_dataset.csv'
eeg_data = pd.read_csv(file_path)
signal_data = eeg_data.iloc[:, :-1].values
labels = eeg_data['label'].values

fs = 500
fc = [0.5, 4, 8, 13, 30, fs / 2]
filter_type = 'dct'
sort_fc = 'ascend'

all_subbands = fdm(signal_data, fs, fc, filter_type=filter_type, sort_fc=sort_fc, plot_subbands=False)

num_samples, _, num_subbands = all_subbands.shape
reshaped_subbands = all_subbands.swapaxes(1, 2).reshape(num_samples, -1)
output_file = 'outputseeg_subbands.csv'
pd.DataFrame(reshaped_subbands).to_csv(output_file, index=False)

print(f"Sub-band data saved to {output_file}")


Sub-band data saved to outputseeg_subbands.csv
